In [1]:
import sys
import os
import numpy as np
import pandas as pd
from PIL import Image

import ipywidgets as widgets
from io import BytesIO

from ganpfinder import GANPfinder
from jupyterGUI import GUIses

StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.


### Description:

Possible datasets: ffhq, car, cat, church, horse, bedrooms, kitchen, places

---

In [2]:
gpf = GANPfinder(class_name="car", gan_sample_seed=0, n_comp_in_use=20)
guises = GUIses(gpf)
guises.run()

Not cached
[29.07 11:11] Computing stylegan2-car_style_ipca_c80_n1000000_w.npz
Reusing InstrumentedModel instance
Using W latent space
Feature shape: torch.Size([1, 512])
B=200, N=1000000, dims=512, N/dims=1953.1


Fitting batches (NB=2000): 100%|##########| 500/500 [11:07<00:00,  1.33s/it]


Total time: 0:16:50.467438


In [ ]:
# CONTINUE

---